In [9]:
# Imports
import pandas as pd
from sqlalchemy import create_engine, text

import bcrypt

In [10]:
# Create a sqlalchemy engine to input database
def get_engine(dbname, host='localhost', port=5432, user='nihar'):
    connection_string = f'postgresql://{user}@{host}:{port}/{dbname}'
    engine = create_engine(connection_string)
    return engine

In [11]:
# The function that runs any query; returns results for SELECT queries and commits changes for non-SELECT queries
def execute_query(dbname, query):
    # Establish the connection
    engine = get_engine(dbname)
    
    # Create transaction
    with engine.connect() as connection:
        transaction = connection.begin() 
        try:
            # If SELECT query, fetch and return the results
            if query.strip().lower().startswith('select'):
                result = connection.execute(text(query))
                return result.fetchall()
            
            # Execute non-SELECT queries
            else:
                connection.execute(text(query))
                # Commit the changes and return None
                transaction.commit()
                return None 
        except Exception as e: # Re-raise the exception
            transaction.rollback()  
            raise e 

In [12]:
def get_hashed_password(password_text):
    # First time hashing password using bcrypt; the salt is saved into the hash itself; slow hash to prevent malicious brute force attacks
    password_bytes = password_text.encode('utf-8')
    return bcrypt.hashpw(password_bytes, bcrypt.gensalt(14))

def check_password(password_text, password_hash):
    # Check hashed password
    password_bytes = password_text.encode('utf-8')
    return bcrypt.checkpw(password_bytes, password_hash)


In [13]:
# Returns user database
def get_users():
    # Query to select all users
    query = "SELECT * FROM users;"
    df = pd.DataFrame(execute_query('users', query))
    return df

In [14]:
def get_id(username):
    # SQL query to get id of user
    query = f"SELECT id FROM users WHERE username = '{username}';"
    id = execute_query('users', query)
    if len(id) == 0: return None  # Username not found in database
    return id[0][0] # Extract data and return

In [15]:
# Functions to add and remove users

def add_user(username, email, password, hostname = 'localhost', port = 5432, postgre_user = 'nihar'):
    # SQL query to insert a new user
    password_hash = get_hashed_password(password).decode('utf-8')
    query = "INSERT INTO users (username, email, password_hash) VALUES ('%s', '%s', '%s');" % (username, email, password_hash)
    execute_query('users', query)

def remove_user(username, hostname = 'localhost', port = 5432, postgre_user = 'nihar'):
    # SQL query to delete a user based on the username
    id = get_id(username)
    if id == None: return
    query = f"DELETE FROM users WHERE id = '{id}';"
    execute_query('users', query)

In [16]:
# Function to validate password of an user 

def validate_password(username, pass_check, hostname = 'localhost', port = 5432, postgre_user = 'nihar'):
    # SQL query to delete a user based on the username
    id = get_id(username)
    if id == None: return
    query = f"SELECT password_hash FROM users WHERE id = '{id}';"
    
    password_hash = execute_query('users', query)[0][0]
    # print(password_hash.tobytes())
    # return check_password(pass_check, password_hash)

In [17]:
# print(validate_password("jethalal", "hello"))

In [18]:
# Update observation helper function
def update_info(id, updated_info, category): 
    if id == None: return # If not a valid user, terminate
    query = f"UPDATE users SET {category} = '{updated_info}' WHERE id = {id};" # SQL query to update a user's info based on id
    execute_query('users', query)

# Update observation's data functions
def update_username(id, new_username):
    update_info(id, new_username, 'username')

def update_email(id, new_email):
    update_info(id, new_email, 'email')

def update_password(id, new_pass):
    new_hash = get_hashed_password(new_pass).decode('utf-8')
    update_info(id, new_hash, 'password_hash')

In [19]:
# Get ID example
print(get_id("jethalal"))

115559616851683


In [20]:
# Password hashing and checking example
password_hash = get_hashed_password("cheese")
print(password_hash)

print(check_password("cheese", password_hash))
print(check_password("burger", password_hash))

b'$2b$14$XDrr6g/KObKdcWaKaFkQJeeiEhpR5ws6TGXBUfFU8qJLegP4ZTREy'
True
False


In [21]:
# Get DB example
get_users()

,id,username,email,password_hash
0,115559616851683,jethalal,sethji@shaadi.com,"[b'$', b'2', b'b', b'$', b'1', b'4', b'$', b'M..."


In [22]:
# Add user example
remove_user("chungus") # Ensure doesn't exist since we're asserting users are unique
add_user("chungus", "bingus@yahoo.co.in", "evillarry")
get_users()

,id,username,email,password_hash
0,115559616851683,jethalal,sethji@shaadi.com,"[b'$', b'2', b'b', b'$', b'1', b'4', b'$', b'M..."
1,846457368193647,chungus,bingus@yahoo.co.in,"[b'$', b'2', b'b', b'$', b'1', b'4', b'$', b'L..."


In [23]:
# Remove user example
remove_user("chungus")
df = get_users()
df

,id,username,email,password_hash
0,115559616851683,jethalal,sethji@shaadi.com,"[b'$', b'2', b'b', b'$', b'1', b'4', b'$', b'M..."


In [24]:
# Update User data example
update_username(get_id("jethalal"), "tapu ke papa")
update_email(get_id("tapu ke papa"), "jijaji@sundar.con")
update_password(get_id("tapu ke papa"), "JalebiMaster")
df = get_users()

# Undo changes
update_username(get_id("tapu ke papa"), "jethalal")
update_email(get_id("jethalal"), "sethji@shaadi.com")
update_password(get_id("jethalal"), "natukaka123")

df

,id,username,email,password_hash
0,115559616851683,tapu ke papa,jijaji@sundar.con,"[b'$', b'2', b'b', b'$', b'1', b'4', b'$', b'5..."


In [ ]:
CREATE TABLE birds (
    id SERIAL PRIMARY KEY,
    name TEXT NOT NULL,
    species TEXT,
    description TEXT
);

CREATE TABLE users_birds (
    user_id INTEGER REFERENCES users(id) ON DELETE CASCADE,
    bird_id INTEGER REFERENCES birds(id) ON DELETE CASCADE,
    PRIMARY KEY (user_id, bird_id)
);

def add_bird(name, species=None, description=None):
    query = f"""
    INSERT INTO birds (name, species, description)
    VALUES ('{name}', '{species or ''}', '{description or ''}');
    """
    execute_query('users', query)

def link_user(username, bird_name):
    user_id = get_id(username)
    if user_id is None: return

    query = f"SELECT id FROM birds WHERE name = '{bird_name}';"
    result = execute_query('users', query)
    if not result: return
    bird_id = result[0][0]

    link_query = f"INSERT INTO users_birds (user_id, bird_id) VALUES ({user_id}, {bird_id}) ON CONFLICT DO NOTHING;"
    execute_query('users', link_query)

def get_users_birds(username):
    user_id = get_id(username)
    if user_id is None: return []
    query = f"""
    SELECT b.name, b.species, b.description
    FROM birds b
    JOIN users_birds ub ON b.id = ub.bird_id
    WHERE ub.user_id = {user_id};
    """
    return pd.DataFrame(execute_query('users', query))

def unlink_bird(username, bird_name):
    user_id = get_id(username)
    if user_id is None: return

    query = f"SELECT id FROM birds WHERE name = '{bird_name}';"
    result = execute_query('users', query)
    if not result: return
    bird_id = result[0][0]

    delete_query = f"DELETE FROM user_birds WHERE user_id = {user_id} AND bird_id = {bird_id};"
    execute_query('users', delete_query)